In [6]:
import json
import random

In [5]:
def read_dataset(file):
    result = []
    with open(file, "r", encoding="utf8") as r:
        for line in r:
            result.append(json.loads(line))
    result.sort(key=lambda x: x["date"])
    return result


train_obj = read_dataset('.\gazeta_dataset\gazeta_train.jsonl')
test_obj = read_dataset('.\gazeta_dataset\gazeta_test.jsonl')
val_obj =  read_dataset('.\gazeta_dataset\gazeta_val.jsonl')
full_obj = train_obj + test_obj + val_obj # для TextRank

In [8]:
def normalize(li_word):
    global normalized_freq
    normalized_freq = []
    for dictionary in li_word:
        max_frequency = max(dictionary.values())
        for word in dictionary.keys():
            dictionary[word] = dictionary[word]/max_frequency
        normalized_freq.append(dictionary)
    return normalized_freq

def calc_word_freq(article_word):
    calc_word_freq = {}
    li_word = []
    for sentence in article_word:
        for word in word_tokenize(sentence):
            if word not in calc_word_freq.keys():
                calc_word_freq[word] = 1
            else:
                calc_word_freq[word] += 1
        li_word.append(calc_word_freq)
        calc_word_freq = {}
    normalize(li_word)
    return normalized_freq

def sentence_score(sentences):
    global sentence_score_list
    sentence_score = {}
    sentence_score_list = []
    for list_, dictionary in zip(sentences, normalized_freq):
        for sent in list_:
            for word in word_tokenize(sent):
                if word in dictionary.keys():
                    if sent not in sentence_score.keys():
                        sentence_score[sent] = dictionary[word]
                    else:
                        sentence_score[sent] += dictionary[word]
        sentence_score_list.append(sentence_score)
        sentence_score = {}
    return sentence_score_list

# Function to tokenize the sentence
def tokenize(article_sent):
    sentence_list = []
    sent_token = []
    for sent in article_sent:
        token = tokenizeize(sent)
        for sentence in token:
            sub_token = ''.join(word for word in sentence if word not in punctuation)
            sub_token = re.sub(' +', ' ', sub_token)
            sent_token.append(sub_token)
        sentence_list.append(sent_token)
        sent_token = []
    sentence_score(sentence_list)
    return sentence_score_list

def summary(sentence_score):
    summary_list = []
    for summ in sentence_score:
        select_length = int(len(summ)*0.25)
        summary_ = nlargest(select_length, summ, key = summ.get)
        summary_list.append(".".join(summary_))
    return summary_list


def to_seris(art):
    global dataframe
    data_dict = {'article' : [art]}
    dataframe = pd.DataFrame(data_dict)['article']
    return dataframe

#Основная функция
def article_summarize(artefact):
    if type(artefact) != pd.Series:
        artefact = to_seris(artefact)
    df = preprocessing(artefact)
    word_normalization = calc_word_freq(df)
    sentence_score = tokenize(article_sent)
    summarized_article = summary(sentence_score)
    return summarized_article

# article_seried = make_series(article)
# article_summarize(article_seried)

In [ ]:
from summa.summarizer import summarize
#библиотечный TextRank, для сравнения
def predict_text_rank(text, summary, summary_part=0.1):
    return summarize(text, ratio=summary_part, language='russian').replace("\n", " ")

calc_method_score(test_obj, predict_text_rank)